In [3]:
# # load dataset

# from datasets import load_dataset
# from tokenizers import ByteLevelBPETokenizer

# tokenizer = ByteLevelBPETokenizer()
# dataset = load_dataset("roneneldan/TinyStories")

# # Specify the split you want to save (e.g., "train", "validation", "test")
# split = "train"

# # Get the desired split from the dataset
# subset = dataset[split]

# # Save the subset to a text file
# subset.to_csv("tinystories-train.txt", sep="\t", index=False)


Repo card metadata block was not found. Setting CardData to empty.


Creating CSV from Arrow format:   0%|          | 0/2120 [00:00<?, ?ba/s]

1919175373

In [1]:
#----- imports --------
import tqdm
import torch
import wandb
import os
import tokenizers


device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
assert device == 'cuda', "This notebook is not optimized for CPU"

config = {
    "learning_rate": 1e-3,
    "eval_interval": 300,
    "max_iters": 3000, 
    "H": 16,
    "B": 64,
    "T": 64,
    "C": 256,
    "feedforward_factor": 3,
    "n_heads": 6,
    "dropout": 0.4,
    "l2_penalty": 0.1,
    "n_layers": 6,
    "tokenizer_vocab_size": 2**15,
    "git_hash": os.popen("git rev-parse HEAD").read().strip()
}

# initial
for k,v in config.items():
    locals ()[k] = v


# wandb.init(
#     project = "tinystories",
#     config = config
# )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: llmhacking. Use `wandb login --relogin` to force relogin


In [17]:

with open('tinystories-train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [11]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1916206969


In [12]:
print("length of dataset in lines: ", len(text.split('\n')))

length of dataset in lines:  20550005


In [13]:
print(text[:1000])

text
"One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, ""Mom, I found this needle. Can you share it with me and sew my shirt?"" Her mom smiled and said, ""Yes, Lily, we can share the needle and fix your shirt.""

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together."
"Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.

One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that wer

In [18]:
paths = ['tinystories-train.txt']
tokenizer = tokenizers.ByteLevelBPETokenizer()

tokenizer.train(files=paths, vocab_size=tokenizer_vocab_size, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

tokenizer.save_model('.', 'tiny-stories-bpe')



enc = tokenizer.encode("She sells sea shells by the sea shore!")
tokenizer.decode(enc.ids)


def encode(text):
    return tokenizer.encode(text).ids
def decode(encoded_text):
    return tokenizer.decode(encoded_text)

hello_encoded = encode("hello")
print(hello_encoded)
print(decode(hello_encoded))
vocab_size = tokenizer.get_vocab_size()




[6137]
hello


In [ ]:
print("length of dataset in characters: ", len(text[:10000]))
print("length of dataset in tokens: ", len(encode(text[:10000])))
chars_per_token = len(text[:10000]) / len(encode(text[:10000]))
print("characters per token: ", chars_per_token)

In [107]:

data = torch.tensor(encode(text), dtype=torch.long)
print(data.dtype)
print(data.size())
print(data.device)


torch.int64
torch.Size([344152])
cuda:0


In [108]:
n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]

In [109]:
train_data[:T+1]

tensor([ 676, 1201,   30,  203, 2347,  336, 2752,  807, 2307,   16,  679,  322,
         621,   18,  203,  203, 1236,   30,  203, 2543,   16,  621,   18,  203,
         203,  676, 1201,   30,  203,  570,  423,  400, 2058,  773, 1503,  292,
         969,  532,  292, 2797,  561,   35,  203,  203, 1236,   30,  203, 3667,
         499,  773,   18, 2058,  773,   18,  203,  203,  676, 1201,   30,  203,
         676,   16,  293,  509, 3997], device='cuda:0')

In [110]:
x = train_data[:T]
y = train_data[1:T+1]
for t in range(T):
    context = x[:t+1]
    target = y[t]
    # print("when we see the text", context, "we predict the next character is", target)

In [111]:
torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, data.size(0) - T, (B,)) # 4 random locations we can sample from
    x = torch.stack([data[i:i+T] for i in ix]) # random sequences
    y = torch.stack([data[i+1:i+T+1] for i in ix]) # next character for each random sequence

    return x, y

xb, yb = get_batch('train')

for b in range(B):
    for t in range(T): # for each of the characters in the sample
        context = xb[b, :t+1]
        target = yb[b, t]


In [112]:

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class Head(nn.Module):
    '''One Head of self-attention'''
    def __init__(self, H):
        super().__init__()
        self.query = nn.Linear(C, H, bias=False)
        self.key = nn.Linear(C, H, bias=False)
        self.value = nn.Linear(C, H, bias=False)
        # self.output = nn.Linear(H, C, bias=False) # output matrix
        self.register_buffer('tril', torch.tril(torch.ones(T, T)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Query and Key matrices for the attention mechanism
        # x: 8 tokens
        # Q: 16 tall (arbitrary), 32 long channels
        # K: 16 tall (arbitrary), 32 long channels

        query_vectors = self.query(x)
        key_vectors = self.key(x)


        # Attention masking(so we can't look into the past):

        tril = self.tril
        wei = torch.zeros(T, T) 
        wei = wei.masked_fill(tril == 0, float('-inf')) # set the upper triangular to -inf
        # xbow = wei @ x # apply the mask to the input, bag of words because simple avg.

        # multiply the two to get the attention weights
        attention_pattern = query_vectors @ key_vectors.transpose(-2, -1) # T, T
        attention_pattern = attention_pattern / (H ** 0.5) # scale the attention pattern for numerical stability
        attention_weights = F.softmax(attention_pattern + wei, dim=-1) # T, T (the row dimension is the query)
        attention_weights = self.dropout(attention_weights)

        value_vectors = self.value(x) # the direction we should go in the embedding space for each token (ie more blue) T, H

        # apply the attention weights to the value vectors
        context = attention_weights @ value_vectors # T, H

        # project back into original space from value space
        # return self.output(context)
        return context

x = torch.randn(B,T,C)
head = Head(H)
# head(x)

In [113]:
class MultiHeadAttention(nn.Module):
    '''Multiple heads of self-attention'''
    def __init__(self, H, C, n_heads): # H is head embedding space size, n_heads is number of heads
        super().__init__()
        self.heads = nn.ModuleList([Head(H) for _ in range(n_heads)])
        self.combine_heads = nn.Linear(H*n_heads, C)
        self.dropout = nn.Dropout(dropout)


    def forward(self,x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.combine_heads(x)  # T, C
        return self.dropout(x)

In [114]:
head = MultiHeadAttention(H, C, n_heads)
head.heads[0].forward(x).shape


torch.Size([64, 64, 16])

In [115]:
class FeedForward(nn.Module):
    '''Feed-forward neural network'''
    def __init__(self, C):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(C, C * feedforward_factor),
            nn.ReLU(),
            nn.Linear(C * feedforward_factor, C),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [116]:
class LayerNorm(nn.Module):
    '''Layer normalization'''
    def __init__(self, C, use_affine=True):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(C)) if use_affine else None
        self.beta = nn.Parameter(torch.zeros(C)) if use_affine else None

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        if self.gamma is not None and self.beta is not None:
            return self.gamma * (x - mean) / (std + 1e-6) + self.beta
        else:
            return (x - mean) / (std + 1e-6)

In [117]:
class Block(nn.Module):
    '''Transformer block'''
    def __init__(self, H, C, n_heads):
        super().__init__()
        self.attention = MultiHeadAttention(H, C, n_heads)
        self.ff = FeedForward(C)
        self.norm1 = LayerNorm(C, use_affine=True)
        self.norm2 = LayerNorm(C, use_affine=True)

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

In [118]:
class GPT(nn.Module):

    def __init__(self, n_layers):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, C) 
        self.position_embedding_table = nn.Embedding(T, C)
        self.lm_head = nn.Linear(C, vocab_size)
        self.layers = nn.ModuleList([Block(H, C, n_heads) for _ in range(n_layers)])
        self.block = nn.ModuleList([Block(H, C, n_heads)])
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embedding_table(idx) # batch_dim, sequence_dim, embedding_dim
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = token_emb + pos_emb # token identities and positions contained

        for layer in self.layers:
            x = layer(x)

        logits = self.lm_head(x) # batch_dim, sequence_dim, vocab_size

        batch_dim, sequence_dim, embedding_dim = logits.size()

        # loss = F.cross_entropy(logits, targets) this won't work because we need 1d logits and 1d targets
        # one-hot-vectors are a line in the x-dimension, so the shape of shape of the logits should be (-1, vocab_size).

        if targets is None:
            return logits, None
        else:
            # a list of all the predictions, reguardles of batch.
            # xdim: probabilities of each character in the vocab (embedding_dim=vocab_size)
            # ydim: all predictions for all batches flattened (batch_dim*sequence_dim)
            logits_loss_view = logits.view(-1, vocab_size) 
            # targets loss view
            # xdim: all targets for all batches flattened (batch_dim*sequence_dim)
            # so this would be like, [1,4,5,1,2,3, ...]
            # where each number is the correct next index of the one hot vector
            targets_loss_view = targets.view(-1)
            loss = F.cross_entropy(logits_loss_view, targets_loss_view)
            return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx[:,-T:])
            # get the predictions of the last token
            last_token_logits = logits[:, -1, :] # all batches, last token, all probabilities
            # softmax to get probabilities
            probabilities = F.softmax(last_token_logits, dim=-1)
            # sample from the probabilities
            next_token = torch.multinomial(probabilities, num_samples=1)
            # add the new token to the idx tensor
            idx = torch.cat((idx, next_token), dim=1)
        return idx

    

model = GPT(n_layers)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)




test_idx = torch.zeros(1, T).long()
model.forward(idx=test_idx)
# decode(model.generate(idx=test_idx, max_new_tokens=100)[0].tolist())

torch.Size([64, 64, 4096])
tensor(8.7672, device='cuda:0', grad_fn=<NllLossBackward0>)


(tensor([[[-2.3841,  1.0758, -1.2788,  ...,  0.8275, -0.7848, -0.0043],
          [-0.6179, -2.2806,  0.1570,  ...,  2.7975, -0.9502,  1.6256],
          [-1.1456,  0.3364,  0.3847,  ...,  1.6403, -1.9459, -0.5270],
          ...,
          [-1.0044, -1.1411, -0.1971,  ...,  1.2776, -0.2699,  1.2927],
          [-2.0422, -0.6010, -0.1549,  ...,  1.0831, -1.7448, -0.0553],
          [-1.4777, -0.6368,  0.0028,  ...,  2.0669, -0.9934,  0.3146]]],
        device='cuda:0', grad_fn=<ViewBackward0>),
 None)

In [119]:
model

GPT(
  (token_embedding_table): Embedding(4096, 192)
  (position_embedding_table): Embedding(64, 192)
  (lm_head): Linear(in_features=192, out_features=4096, bias=True)
  (layers): ModuleList(
    (0-5): 6 x Block(
      (attention): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (query): Linear(in_features=192, out_features=16, bias=False)
            (key): Linear(in_features=192, out_features=16, bias=False)
            (value): Linear(in_features=192, out_features=16, bias=False)
            (dropout): Dropout(p=0.4, inplace=False)
          )
        )
        (combine_heads): Linear(in_features=96, out_features=192, bias=True)
        (dropout): Dropout(p=0.4, inplace=False)
      )
      (ff): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=192, out_features=768, bias=True)
          (1): ReLU()
          (2): Linear(in_features=768, out_features=192, bias=True)
          (3): Dropout(p=0.4, inplace=False)
      

In [120]:
# logits, loss = self(idx[:,-T:])

idx = torch.zeros(1, 1).long()
idx[:,-T:]

tensor([[0]], device='cuda:0')

In [121]:
model.token_embedding_table.weight.device

device(type='cuda', index=0)

In [122]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



In [123]:
eval_iters = 10
eval_interval = 300
@torch.no_grad()
def estimate_loss(is_last=False):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        real_iters = eval_iters
        if is_last and split == 'val':  # increase last eval to mitigate noise
            real_iters *= 10 
        losses = torch.zeros(real_iters)
        for k in range(real_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean() / chars_per_token
    model.train()
    return out
    

In [124]:
# get the number of parameters
n_params = sum(p.numel() for p in model.parameters())
parameter_to_data_ratio = n_params / len(train_data)
print(f"{parameter_to_data_ratio=}")

parameters = []
for name, param in model.named_parameters():
    parameters.append({"name": name, "params": param.numel()})

# sort parameters by size
sorted_parameters = sorted(parameters, key=lambda x: x["params"], reverse=True)
for p in sorted_parameters:
    print(f"{p['name']}: {p['params']}")

parameter_to_data_ratio=13.505591858873363
token_embedding_table.weight: 786432
lm_head.weight: 786432
layers.0.ff.net.0.weight: 147456
layers.0.ff.net.2.weight: 147456
layers.1.ff.net.0.weight: 147456
layers.1.ff.net.2.weight: 147456
layers.2.ff.net.0.weight: 147456
layers.2.ff.net.2.weight: 147456
layers.3.ff.net.0.weight: 147456
layers.3.ff.net.2.weight: 147456
layers.4.ff.net.0.weight: 147456
layers.4.ff.net.2.weight: 147456
layers.5.ff.net.0.weight: 147456
layers.5.ff.net.2.weight: 147456
block.0.ff.net.0.weight: 147456
block.0.ff.net.2.weight: 147456
layers.0.attention.combine_heads.weight: 18432
layers.1.attention.combine_heads.weight: 18432
layers.2.attention.combine_heads.weight: 18432
layers.3.attention.combine_heads.weight: 18432
layers.4.attention.combine_heads.weight: 18432
layers.5.attention.combine_heads.weight: 18432
block.0.attention.combine_heads.weight: 18432
position_embedding_table.weight: 12288
lm_head.bias: 4096
layers.0.attention.heads.0.query.weight: 3072
layer

In [125]:
num_params = sum([p.numel() for p in model.parameters()])

for steps in tqdm.tqdm(range(max_iters)):
    xb, yb = get_batch('train')
    # loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    # l2 regularization
    l2 = sum(p.pow(2).sum() for p in model.parameters()) / num_params
    loss = loss + l2 * l2_penalty

    loss.backward()
    optimizer.step()
    if steps % eval_interval == 0:
        losses = estimate_loss()
        wandb.log({"train": losses['train'].item(), "val": losses['val'].item(), "l2":l2})

losses = estimate_loss(is_last=True)
wandb.log({"train": losses['train'].item(), "val": losses['val'].item()})
wandb.finish()


100%|██████████| 3000/3000 [01:16<00:00, 39.16it/s]


l2,█▄▃▂▁▁▁▁▁▂
train,█▃▃▂▂▂▂▁▁▁▁
val,█▂▁▁▁▁▁▁▁▁▁
l2,0.17729
train,0.9381
val,1.43439


In [126]:
estimate_loss()

{'train': tensor(0.9418, device='cuda:0'),
 'val': tensor(1.4320, device='cuda:0')}

In [131]:
test_idx = torch.zeros(1, T).long()
print(decode(
    model.generate(idx=test_idx, max_new_tokens=C)[0].tolist()
)[T:])

my lord a precious queen
To visiting me to bring me from this haste
Than that my spring enrichised him said with such a battle
Forget, in aptier'd the deputy of this:
What valiant Richard doth he sends the thigh.
Romeo I hand thy thoughts to permit to wonder the God
That will win him.

GLOUCESTER:
What would theseBISHOP OF YORK:
O me, cousin, the painful dog is here.
I'll find out-night never spake thy wretched hay
And spinking cloy's with heavy eyes;
Bid much excuse thy false dearly doth beguem,
Why should thrown me with batt the heart?
And not in the golden fee with thy lips,
Nor no tongue, love his life
